In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [3]:
crimeDF= pd.read_csv("Seattle_crime.csv", low_memory=False)
weatherDF = pd.read_csv("Seattle_weather.csv")

In [4]:
crimeDF.head()

,Report Number,Offense Start DateTime,Offense End DateTime,Report DateTime,Crime Against Category,Offense Parent Group,Offense
0,2020-044620,2/5/2020 10:10,NaN,2/5/2020 11:24,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations
1,2020-044452,2/3/2020 8:00,2/4/2020 8:00,2/5/2020 10:06,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories
2,2020-044465,2/2/2020 20:30,2/2/2020 21:30,2/5/2020 9:39,PROPERTY,ROBBERY,Robbery
3,2020-044225,2/5/2020 1:17,2/5/2020 2:21,2/5/2020 3:30,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property
4,2020-044076,2/5/2020 0:51,NaN,2/5/2020 0:51,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence


In [5]:
weatherDF.head()

,DATE,PRCP,TMAX,TMIN,RAIN,TAVG
0,12/14/2017,0.0,50.0,36.0,False,43.0
1,12/13/2017,0.0,48.0,34.0,False,41.0
2,12/12/2017,0.0,46.0,32.0,False,39.0
3,12/11/2017,0.0,49.0,29.0,False,39.0
4,12/10/2017,0.0,49.0,34.0,False,41.5
